In [6]:
#import libraries

import requests
import pandas as pd
import time
import credentials

In [7]:
key = credentials.youtube

In [9]:
url_video = 'https://www.googleapis.com/youtube/v3/search?key='+key+'&part=snippet&channelId=UCrPseYLGpNygVi34QpGNqpA&order=date&maxResults=1000&'

video_api = requests.get(url_video).json()

In [10]:
def get_video_stats(video_id):
    url_stats = 'https://www.googleapis.com/youtube/v3/videos?key='+key+'&part=statistics&id='+video_id
    stats_api = requests.get(url_stats).json()

    views = stats_api['items'][0]['statistics']['viewCount']
    likes = stats_api['items'][0]['statistics']['likeCount']
    comments = stats_api['items'][0]['statistics']['commentCount']

    return views, likes, comments

In [13]:
def get_video(df): #passing dataframe as a parameter
    pageToken = ''
    while 1:
        url_video = 'https://www.googleapis.com/youtube/v3/search?key='+key+'&channelId=UCrPseYLGpNygVi34QpGNqpA&part=snippet&order=date&maxResults=1000'+pageToken
        video_api = requests.get(url_video).json()

        time.sleep(1.5)
        for video in video_api['items'] :
            if video_api['kind'] == 'youtube#searchListResponse' :
                channel_name = video['snippet']['channelTitle']
                video_title = video['snippet']['title']
                video_id = video['id']['videoId']
                
                views, likes, comments = get_video_stats(video_id)

                temp_df = pd.DataFrame({
                'channel_name' : [channel_name],
                'video_title' : [video_title],
                'video_id' : [video_id],
                'views' : [views],
                'likes' : [likes],
                'comments' : [comments]})

                df = pd.concat([df, temp_df], ignore_index=True)

            try:
                if video_api['nextPageToken'] != None:
                    pageToken = '&pageToken=' + video_api['nextPageToken']
            except:
                break

    return df

In [14]:
df = pd.DataFrame(columns=['channel_name','video_title','video_id','views','likes','comments','pos_sentiment','neutral','neg_sentiment'])

df2 = get_video(df)

KeyError: 'items'

In [ ]:
for video in video_api['items'] :
    #time.sleep(1)
    if video_api['items'][0]['id']['kind'] == 'youtube#video' :
        channel_name = video['snippet']['channelTitle']
        video_title = video['snippet']['title']
        video_id = video['id']['videoId']

        views, likes, comments = get_video_stats(video_id)

        channel_metrics = pd.DataFrame({
        'channel_name' : [channel_name],
        'video_title' : [video_title],
        'video_id' : [video_id],
        'views' : [views],
        'likes' : [likes],
        'comments' : [comments]
        })

        metrics = pd.concat([metrics, channel_metrics], ignore_index=True)

    try:
        if video_api['nextPageToken'] != None:
            pageToken = 'pageToken=' + video_api['nextPageToken']
    except:
        break

In [ ]:
metrics2.info()